# Scraping APIs

A site with an API (Application Programming Interface) wants you to scrape it.

Examples abound:

* <a href="https://www.census.gov/data/developers/data-sets.html">U.S. Census APIs</a>
* <a href="https://apps.fas.usda.gov/opendataweb/home">US Agriculture Commodities and Exports</a>
* <a href="https://www.federalregister.gov/developers/documentation/api/v1">Federal Register</a>
* <a href="https://developer.dol.gov/beginners-guide/">Labor Department</a>
* <a href="https://www.eia.gov/">Labor Department</a>

Government sites tend  ```CSVs``` for download but their APIs offer more detailed options for data. They are not trying to hide the data.

Private sites might have APIs, but often charge heafty prices for access beyond a basic number of downloads.

The toughest/hardest part of scraping an API is that they ***ALL HAVE DIFFERENT INSTRUCTIONS*** on how to tap their data.

Today, we'll explore different APIs that each build a different skill:

1. Census health data – **building a simple API call.**
2. USDA commodities exports – **using an API key and targeting specific commodities over several years.**
3. Federal Register – **tapping search terms.**
4. Energy Information Administration – **dealing with pagination.**

What they all have in common:

1. a base url
2. a query string
3. tied together with a query character ```?```
4. an API key.

Combined together these are known as an ```API endpoint```.

You make an ```API call``` (a request) using the ```API endpoint```.




In [ ]:
pip install icecream

In [1]:
## import libraries
import requests
import pandas as pd
from icecream import ic



### 1. Census health data – **building a simple API call.**

- <a href="https://www.census.gov/data/developers/data-sets/Health-Insurance-Statistics.html">Census health landing page</a>
- List of <a href="https://api.census.gov/data/timeseries/healthins/sahie/variables.html">possible variables</a>

We want to create a dataframe with the following info for every state in 2021:

1. Total number insured
2. Percent insured
3. Total number uninsured
4. Percent uninsured
5. Broken down by race

## the parts to build your API endpoint.

In [74]:
## create a dictionary to know what codes mean
target_dict = {
    "NIC_PT": "total_insured",
    "PCTIC_PT": "pct_insured_est",
    "NUI_PT": "total_uninsured",
    "PCTUI_PT": "pct_uninsured_est",
    "RACECAT": "race_category",
    "RACE_DESC": "race_description",
    "STABREV": "state"
    
}

In [75]:
## keys() to get keys
list(target_dict.keys())

['NIC_PT', 'PCTIC_PT', 'NUI_PT', 'PCTUI_PT', 'RACECAT', 'RACE_DESC', 'STABREV']

In [76]:
## values() to get val;;ues
list(target_dict.values())

['total_insured',
 'pct_insured_est',
 'total_uninsured',
 'pct_uninsured_est',
 'race_category',
 'race_description',
 'state']

In [77]:
## format into api query format
my_vars = ",".join(target_dict.keys())
my_vars

'NIC_PT,PCTIC_PT,NUI_PT,PCTUI_PT,RACECAT,RACE_DESC,STABREV'

In [78]:
## endpoint
base_url = "https://api.census.gov/data/timeseries/healthins/sahie?get="
target_year = "2021"

In [79]:
## create query string
query_str = f"{my_vars}&for=state:*&time={target_year}"
query_str

'NIC_PT,PCTIC_PT,NUI_PT,PCTUI_PT,RACECAT,RACE_DESC,STABREV&for=state:*&time=2021'

In [80]:
## create full API call
endpoint = base_url + query_str
endpoint

'https://api.census.gov/data/timeseries/healthins/sahie?get=NIC_PT,PCTIC_PT,NUI_PT,PCTUI_PT,RACECAT,RACE_DESC,STABREV&for=state:*&time=2021'

In [81]:
## get response
response = requests.get(endpoint)



In [84]:
## turn response into json
data = response.json()
data

[['NIC_PT',
  'PCTIC_PT',
  'NUI_PT',
  'PCTUI_PT',
  'RACECAT',
  'RACE_DESC',
  'STABREV',
  'time',
  'state'],
 ['3548525', '88.3', '469887', '11.7', '0', 'All Races', 'AL', '2021', '01'],
 ['2293718',
  '90.0',
  '254385',
  '10.0',
  '1',
  'White alone, not Hispanic or Latino',
  'AL',
  '2021',
  '01'],
 ['921007',
  '87.1',
  '135939',
  '12.9',
  '2',
  'Black or African American alone, not Hispanic or Latino',
  'AL',
  '2021',
  '01'],
 ['163769',
  '73.3',
  '59615',
  '26.7',
  '3',
  'Hispanic or Latino (any race)',
  'AL',
  '2021',
  '01'],
 ['13274',
  '83.7',
  '2587',
  '16.3',
  '4',
  'American Indian and Alaska Native alone, not Hispanic or Latino',
  'AL',
  '2021',
  '01'],
 ['57789',
  '89.8',
  '6551',
  '10.2',
  '5',
  'Asian alone, not Hispanic or Latino',
  'AL',
  '2021',
  '01'],
 ['1784',
  '84.9',
  '317',
  '15.1',
  '6',
  'Native Hawaiian and Other Pacific Islander alone, not Hispanic or Latino',
  'AL',
  '2021',
  '01'],
 ['97184',
  '90.3',
  '1

In [87]:
## create dataframe
df = pd.DataFrame(data[1:], columns =data[0])
df

,NIC_PT,PCTIC_PT,NUI_PT,PCTUI_PT,RACECAT,RACE_DESC,STABREV,time,state
0,3548525,88.3,469887,11.7,0,All Races,AL,2021,01
1,2293718,90.0,254385,10.0,1,"White alone, not Hispanic or Latino",AL,2021,01
2,921007,87.1,135939,12.9,2,"Black or African American alone, not Hispanic ...",AL,2021,01
3,163769,73.3,59615,26.7,3,Hispanic or Latino (any race),AL,2021,01
4,13274,83.7,2587,16.3,4,"American Indian and Alaska Native alone, not H...",AL,2021,01
...,...,...,...,...,...,...,...,...,...
403,248,79.0,66,21.0,6,Native Hawaiian and Other Pacific Islander alo...,WY,2021,56
404,11765,84.3,2195,15.7,7,"Two or More Races, not Hispanic or Latino",WY,2021,56
405,1027525,92.8,79714,7.2,2,"Black or African American alone, not Hispanic ...",PA,2021,42
406,327520,92.8,25315,7.2,7,"Two or More Races, not Hispanic or Latino",OH,2021,39


In [91]:
## rename column headers with more meaning full headers
df.rename(columns = target_dict, inplace = True)
df

,total_insured,pct_insured_est,total_uninsured,pct_uninsured_est,race_category,race_description,state,time,state
0,3548525,88.3,469887,11.7,0,All Races,AL,2021,01
1,2293718,90.0,254385,10.0,1,"White alone, not Hispanic or Latino",AL,2021,01
2,921007,87.1,135939,12.9,2,"Black or African American alone, not Hispanic ...",AL,2021,01
3,163769,73.3,59615,26.7,3,Hispanic or Latino (any race),AL,2021,01
4,13274,83.7,2587,16.3,4,"American Indian and Alaska Native alone, not H...",AL,2021,01
...,...,...,...,...,...,...,...,...,...
403,248,79.0,66,21.0,6,Native Hawaiian and Other Pacific Islander alo...,WY,2021,56
404,11765,84.3,2195,15.7,7,"Two or More Races, not Hispanic or Latino",WY,2021,56
405,1027525,92.8,79714,7.2,2,"Black or African American alone, not Hispanic ...",PA,2021,42
406,327520,92.8,25315,7.2,7,"Two or More Races, not Hispanic or Latino",OH,2021,39


### 2. USDA commodities exports – **using an API key and targeting specific commodities over several years.**

- <a href="https://apps.fas.usda.gov/opendataweb/home">USDA APIs endpoints</a>
- Get an <a href="https://apps.fas.usda.gov/opendataweb/home">API key</a>

We want to create a dataframe with exports between 2020-2022 to all countries for the following commodities:

1. All Wheat
2. Oats
3. Cuts of Beef
4. Cuts of Pork

In [ ]:
## find the parts to build your API call

In [10]:
## get commodities list
com_url = "https://apps.fas.usda.gov/OpenData/api/esr/commodities"

In [11]:
## we create a headers with your API key- use your own
headers = {
    'API_KEY': 'ADD-YOUR-KEY'
}

In [12]:
## now let's put into get requests
## we check the response status code
response  = requests.get(url = com_url, headers = poop)
response.status_code
all_commodities = response.json()
all_commodities

[{'commodityCode': 101, 'commodityName': 'Wheat - HRW', 'unitId': 1},
 {'commodityCode': 102, 'commodityName': 'Wheat - SRW', 'unitId': 1},
 {'commodityCode': 103, 'commodityName': 'Wheat - HRS', 'unitId': 1},
 {'commodityCode': 104, 'commodityName': 'Wheat - White', 'unitId': 1},
 {'commodityCode': 105, 'commodityName': 'Wheat - Durum', 'unitId': 1},
 {'commodityCode': 106, 'commodityName': 'Wheat - Mixed', 'unitId': 1},
 {'commodityCode': 107, 'commodityName': 'All Wheat', 'unitId': 1},
 {'commodityCode': 201, 'commodityName': 'Wheat Products', 'unitId': 1},
 {'commodityCode': 301, 'commodityName': 'Barley', 'unitId': 1},
 {'commodityCode': 401, 'commodityName': 'Corn', 'unitId': 1},
 {'commodityCode': 501, 'commodityName': 'Rye', 'unitId': 1},
 {'commodityCode': 601, 'commodityName': 'Oats', 'unitId': 1},
 {'commodityCode': 701, 'commodityName': 'Sorghum', 'unitId': 1},
 {'commodityCode': 801, 'commodityName': 'Soybeans', 'unitId': 1},
 {'commodityCode': 901, 'commodityName': 'Soybe

### Get endpoint and test out on a single commodity


In [41]:
## your end point here
## trying on all wheat 

endpoint = "https://apps.fas.usda.gov/OpenData/api/esr/exports/commodityCode/107/allCountries/marketYear/2020"
# countries = "https://apps.fas.usda.gov/OpenData/api/esr/countries"

In [44]:
## now let's put into get requests
## we check the response status code
response = requests.get(url = endpoint, headers = headers)
response.status_code

200

In [45]:
## let's store our response into an object called data
response.json()

[{'commodityCode': 107,
  'countryCode': 1220,
  'weeklyExports': 199,
  'accumulatedExports': 199,
  'outstandingSales': 18885,
  'grossNewSales': 11268,
  'currentMYNetSales': 10391,
  'currentMYTotalCommitment': 19084,
  'nextMYOutstandingSales': 0,
  'nextMYNetSales': 0,
  'unitId': 1,
  'weekEndingDate': '2019-06-06T00:00:00'},
 {'commodityCode': 107,
  'countryCode': 2010,
  'weeklyExports': 71970,
  'accumulatedExports': 71970,
  'outstandingSales': 688745,
  'grossNewSales': 365035,
  'currentMYNetSales': 90031,
  'currentMYTotalCommitment': 760715,
  'nextMYOutstandingSales': 0,
  'nextMYNetSales': 0,
  'unitId': 1,
  'weekEndingDate': '2019-06-06T00:00:00'},
 {'commodityCode': 107,
  'countryCode': 2050,
  'weeklyExports': 3300,
  'accumulatedExports': 3300,
  'outstandingSales': 15600,
  'grossNewSales': 8800,
  'currentMYNetSales': 300,
  'currentMYTotalCommitment': 18900,
  'nextMYOutstandingSales': 0,
  'nextMYNetSales': 0,
  'unitId': 1,
  'weekEndingDate': '2019-06-06T0

In [19]:
## convert that list of dicts into a dataframe called df
df = pd.DataFrame(response.json())
df

,commodityCode,countryCode,weeklyExports,accumulatedExports,outstandingSales,grossNewSales,currentMYNetSales,currentMYTotalCommitment,nextMYOutstandingSales,nextMYNetSales,unitId,weekEndingDate
0,107,1220,199,199,18885,11268,10391,19084,0,0,1,2019-06-06T00:00:00
1,107,2010,71970,71970,688745,365035,90031,760715,0,0,1,2019-06-06T00:00:00
2,107,2050,3300,3300,15600,8800,300,18900,0,0,1,2019-06-06T00:00:00
3,107,2110,6600,6600,23000,16551,600,29600,0,0,1,2019-06-06T00:00:00
4,107,2150,0,0,112056,9278,0,112056,0,0,1,2019-06-06T00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
3272,107,7740,0,19650,0,0,0,19650,0,0,1,2020-06-04T00:00:00
3273,107,7870,7250,11150,0,50,7250,11150,0,0,1,2020-06-04T00:00:00
3274,107,7910,44000,80324,0,44000,44000,80324,13000,-40000,1,2020-06-04T00:00:00
3275,107,7920,7150,13150,0,650,7150,13150,0,0,1,2020-06-04T00:00:00


In [22]:
## Now iterate through all our target items

my_commodities = [
    {'commodityCode': 107, 'commodityName': 'All Wheat', 'unitId': 1},
    {'commodityCode': 601, 'commodityName': 'Oats', 'unitId': 1},
    {'commodityCode': 1701, 'commodityName': 'Beef',
  'unitId': 1},
    {'commodityCode': 1702, 'commodityName': 'Pork',
  'unitId': 1}]

my_commodities

[{'commodityCode': 107, 'commodityName': 'All Wheat', 'unitId': 1},
 {'commodityCode': 601, 'commodityName': 'Oats', 'unitId': 1},
 {'commodityCode': 1701, 'commodityName': 'Beef', 'unitId': 1},
 {'commodityCode': 1702, 'commodityName': 'Pork', 'unitId': 1}]

In [25]:
my_commodities[0].get('commodityCode')

107

In [23]:
## endpoint templates
start_endpoint = "https://apps.fas.usda.gov/OpenData/api/esr/exports/commodityCode/"
end_endpoint = "/allCountries/marketYear/"

In [35]:
## iterate to get all the data
target_data = []
failed_endpoints = []
for commodity in my_commodities:
    target_code = commodity.get('commodityCode')
    ic(target_code)
    for year in range(2020,2023):
        ic(year)
        try:
            endpoint = f"{start_endpoint}{target_code}{end_endpoint}{year}"
            ##get response
            response = requests.get(url = endpoint, headers = headers)
            ## turn into dataframe
#             df = pd.DataFrame(response.json)
            target_data.append(pd.DataFrame(response.json()))
        except:
            print(f"Failed to retrieve data from {endpoint}...got status code {response.status_code}")
            failed_endpoints.append(endpoint)
print("done scraping API")    

ic| target_code: 107
ic| year: 2020
ic| year: 2021
ic| year: 2022
ic| target_code: 601
ic| year: 2020
ic| year: 2021
ic| year: 2022
ic| target_code: 1701
ic| year: 2020
ic| year: 2021
ic| year: 2022
ic| target_code: 1702
ic| year: 2020
ic| year: 2021
ic| year: 2022


In [37]:
## call list
len(target_data)

12

In [47]:
## concat into single df
df = pd.concat(target_data).reset_index(drop = True)

In [48]:
## call df
df

,commodityCode,countryCode,weeklyExports,accumulatedExports,outstandingSales,grossNewSales,currentMYNetSales,currentMYTotalCommitment,nextMYOutstandingSales,nextMYNetSales,unitId,weekEndingDate
0,107,1220,199,199,18885,11268,10391,19084,0,0,1,2019-06-06T00:00:00
1,107,2010,71970,71970,688745,365035,90031,760715,0,0,1,2019-06-06T00:00:00
2,107,2050,3300,3300,15600,8800,300,18900,0,0,1,2019-06-06T00:00:00
3,107,2110,6600,6600,23000,16551,600,29600,0,0,1,2019-06-06T00:00:00
4,107,2150,0,0,112056,9278,0,112056,0,0,1,2019-06-06T00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
30828,1702,9110,0,100,53,0,0,153,56,2,1,2023-01-05T00:00:00
30829,1702,9350,0,78,0,0,0,78,0,0,1,2023-01-05T00:00:00
30830,1702,9510,7,154,0,7,7,154,0,0,1,2023-01-05T00:00:00
30831,1702,9610,23,288,0,23,23,288,0,0,1,2023-01-05T00:00:00


In [49]:
## confirm we have all our target commodities
list(df["commodityCode"].unique())

[107, 601, 1701, 1702]

### 3. Federal Register – **tapping search terms.**

We have decades of <a href="https://docs.google.com/spreadsheets/d/130WeumbMZjcoRP4D-1uJ7bM0aKBZzt4N/edit?usp=sharing&ouid=112307892189798608417&rtpof=true&sd=true">SBA Excel files</a> that detail loans given to small businesses to recover after climate disasters. The only information we have about the type of disasters are codes in one of the columns that look like:

- CA-00279
- IL-00051
- NC-00099

The <a href="https://www.federalregister.gov/">Federal Register</a> allows us to search for what these codes stand for. But we can't search for nearly a thousand such disaster codes. When we try to scrape the site, it warns us to use the API instead.

Federal Register <a href="https://www.federalregister.gov/developers/documentation/api/v1#/Federal%20Register%20Documents/get_documents__format_">API documentation</a>

In [ ]:
## find the end point

#### Test on single endpoint after figuring out how to build API call

In [ ]:
## endpoint


In [ ]:
## get data


In [ ]:
## type


In [ ]:
## targeting incidents


### Iterate through entire list of codes

In [ ]:
## Normally will take from df as a list
## build disaster code list
disaster_codes = ["CA-00279","IL-00051", "NC-00099" ]

In [ ]:
## provide base url


In [ ]:
## iterate through all endpoints


In [ ]:
## call list


### 4. Energy Information Administration – **dealing with pagination.**

From the <a href="https://www.eia.gov/">Energy Information Administration</a>, we want to compile energy generation by type of fuel and region for about 5 days.

We will encounter a limit on the number of items per API call.

Find our API endpoint first.

In [ ]:
## your target endpoint


In [ ]:
## get response


In [ ]:
## import ceiling division from math



In [ ]:
## paginate our API calls



In [ ]:
## for loop with pagination


In [ ]:
## call all data


In [ ]:
## length


In [ ]:
## use itertools to flatten list with nested lists


In [ ]:
## flatten nested lists


In [ ]:
## convert to df


In [ ]:
fips_codes = {
    'Alabama': '01',
    'Alaska': '02',
    'Arizona': '04',
    'Arkansas': '05',
    'California': '06',
    'Colorado': '08',
    'Connecticut': '09',
    'Delaware': '10',
    'Florida': '12',
    'Georgia': '13',
    'Hawaii': '15',
    'Idaho': '16',
    'Illinois': '17',
    'Indiana': '18',
    'Iowa': '19',
    'Kansas': '20',
    'Kentucky': '21',
    'Louisiana': '22',
    'Maine': '23',
    'Maryland': '24',
    'Massachusetts': '25',
    'Michigan': '26',
    'Minnesota': '27',
    'Mississippi': '28',
    'Missouri': '29',
    'Montana': '30',
    'Nebraska': '31',
    'Nevada': '32',
    'New Hampshire': '33',
    'New Jersey': '34',
    'New Mexico': '35',
    'New York': '36',
    'North Carolina': '37',
    'North Dakota': '38',
    'Ohio': '39',
    'Oklahoma': '40',
    'Oregon': '41',
    'Pennsylvania': '42',
    'Rhode Island': '44',
    'South Carolina': '45',
    'South Dakota': '46',
    'Tennessee': '47',
    'Texas': '48',
    'Utah': '49',
    'Vermont': '50',
    'Virginia': '51',
    'Washington': '53',
    'West Virginia': '54',
    'Wisconsin': '55',
    'Wyoming': '56',
    'District of Columbia': '11',
    'Puerto Rico': '72'
}
